In [28]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv('/home/users/akshay/PCPpred/data/CycPeptMPDB_Peptide_Assay_PAMPA (5).csv')
df.shape

(7298, 247)

In [18]:
last_id = df['ID'].max()
print('last_id: ', last_id)

last_id:  8466


In [19]:
#Step 1: Removing unreliable datapoints (datapoints with Permeability value <= -10) and selecting the required Columns
df = df[df['Permeability'] > -10]
selected_columns = ['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']
new_df = df[selected_columns]
new_df.shape

(7027, 5)

In [20]:
new_df.head(10)

,ID,SMILES,Permeability,Sequence,MolWt
0,1,C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...,-6.6,"['Abu', 'Sar', 'meL', 'V', 'meL', 'A', 'dA', '...",1202.635
1,2,CC(C)C[C@@H]1NC(=O)[C@@H](CC(C)C)NC(=O)[C@@H](...,-6.2,"['dL', 'dL', 'L', 'dL', 'P', 'Y']",712.933
2,3,CC(C)C[C@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C@...,-7.0,"['dL', 'dL', 'dL', 'dL', 'P', 'Y']",712.933
3,4,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc...,-7.1,"['L', 'L', 'L', 'dL', 'P', 'Y']",712.933
4,5,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...,-7.2,"['L', 'dL', 'dL', 'dL', 'P', 'Y']",712.933
5,6,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc...,-7.3,"['L', 'L', 'L', 'L', 'dP', 'Y']",712.933
6,7,CC(C)C[C@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C@...,-7.3,"['dL', 'dL', 'dL', 'dL', 'dP', 'Y']",712.933
7,8,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...,-7.3,"['L', 'L', 'dL', 'dL', 'P', 'Y']",712.933
8,9,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...,-8.1,"['L', 'dL', 'L', 'dL', 'dP', 'Y']",712.933
9,10,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...,-8.1,"['L', 'dL', 'L', 'L', 'dP', 'Y']",712.933


In [21]:
#Step 2: curating duplicate dapaoints based on SMILES representation
duplicates = new_df[new_df.duplicated(subset='SMILES', keep=False)]
non_duplicates = new_df[~new_df.duplicated(subset='SMILES', keep=False)]
print("DataFrame with duplicates:")
print(duplicates)
print("\nDataFrame without duplicates:")
print(non_duplicates)
duplicates.to_csv('/home/users/akshay/PCPpred/data/duplicates.csv', index=False)

DataFrame with duplicates:
        ID                                             SMILES  Permeability  \
0        1  C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...     -6.600000   
1        2  CC(C)C[C@@H]1NC(=O)[C@@H](CC(C)C)NC(=O)[C@@H](...     -6.200000   
4        5  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...     -7.200000   
8        9  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...     -8.100000   
9       10  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...     -8.100000   
...    ...                                                ...           ...   
7263  8432  CCC[C@H]1C(=O)N[C@@H](CC2CCCCC2)C(=O)N(C)[C@@H...     -5.710382   
7264  8433  CCC[C@H]1C(=O)N[C@@H](CC2CCCCC2)C(=O)N(C)[C@@H...     -5.602181   
7265  8434  CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](CC2C...     -5.675509   
7266  8435  CCC[C@H]1C(=O)N[C@@H](CC2CCCCC2)C(=O)N(C)[C@@H...     -5.962810   
7267  8436  CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](CC2C...     -6.323721   

                        

In [23]:
duplicates = new_df[new_df.duplicated(subset='SMILES', keep=False)]

non_duplicates = new_df[~new_df.duplicated(subset='SMILES', keep=False)]
#Permeability and MolWt of duplicate SMILES were averaged, and Most abundandant sequence was chosen
grouped_duplicates = duplicates.groupby('SMILES').agg(
    Permeability=('Permeability', 'mean'),   
    MolWt=('MolWt', 'mean'),   
    Sequence=('Sequence', lambda x: x.mode()[0]), 
).reset_index()

#Unique ID's were given to these curated datapoints, where ID's start from the last documented id in CycPeptMPDB_Peptide_Assay_PAMPA (5)
grouped_duplicates['ID'] = range(last_id + 1, last_id + 1 + len(grouped_duplicates))

grouped_duplicates = grouped_duplicates[['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']]

non_duplicates_ordered = non_duplicates[['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']]
merged_df = pd.concat([non_duplicates_ordered, grouped_duplicates], ignore_index=True)

print("DataFrame with processed duplicates:")
print(grouped_duplicates)

print("\nDataFrame without duplicates:")
print(non_duplicates)

print("Merged DataFrame:")
print(merged_df)

grouped_duplicates.to_csv('/home/users/akshay/PCPpred/data/grouped_duplicates.csv', index=False)
non_duplicates.to_csv('/home/users/akshay/PCPpred/data/non_duplicates.csv', index=False)
merged_df.to_csv('/home/users/akshay/PCPpred/data/CycPeptMPDB_data_Main.csv', index=False)

DataFrame with processed duplicates:
      ID                                             SMILES  Permeability  \
0   8467  C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](C(...     -5.745000   
1   8468  C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](C)...     -5.485000   
2   8469  C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...     -5.851667   
3   8470  C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC...     -5.655000   
4   8471  C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H]([C...     -5.950000   
5   8472  C/C=C/C[C@@H](C)[C@H]1OC(=O)[C@H](C(C)C)N(C)C(...     -6.000000   
6   8473  CC(=O)N1CCC[C@H]1C(=O)N(C)[C@H](CC(C)C)C(=O)N[...     -5.200000   
7   8474  CC(C)C[C@@H]1NC(=O)CNC(=O)[C@@H]2CCCN2C(=O)[C@...     -7.445000   
8   8475  CC(C)C[C@@H]1NC(=O)[C@@H](CC(C)C)N(C)C(=O)[C@H...     -5.960000   
9   8476  CC(C)C[C@@H]1NC(=O)[C@@H](CC(C)C)NC(=O)[C@@H](...     -5.830000   
10  8477  CC(C)C[C@@H]1NC(=O)[C@@H](CC(C)C)NC(=O)[C@H](C...     -5.715000   
11  8478  CC(C)C[C@@H]1NC(=O)[C@@H](CC(

In [ ]:
#Step 3: Splitting the Main dataset into Train and Test in ration 0.8:0.2

In [29]:
df = pd.read_csv('/home/users/akshay/PCPpred/data/CycPeptMPDB_data_Main.csv')
df

,ID,SMILES,Permeability,Sequence,MolWt
0,3,CC(C)C[C@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C@...,-7.000000,"['dL', 'dL', 'dL', 'dL', 'P', 'Y']",712.933
1,4,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc...,-7.100000,"['L', 'L', 'L', 'dL', 'P', 'Y']",712.933
2,6,CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc...,-7.300000,"['L', 'L', 'L', 'L', 'dP', 'Y']",712.933
3,7,CC(C)C[C@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C@...,-7.300000,"['dL', 'dL', 'dL', 'dL', 'dP', 'Y']",712.933
4,8,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...,-7.300000,"['L', 'L', 'dL', 'dL', 'P', 'Y']",712.933
...,...,...,...,...,...
6955,8514,CCC[C@H]1C(=O)N[C@@H](CC2CCCCC2)C(=O)N(C)[C@@H...,-5.929053,"['Chg', 'meA', 'Cha', 'dP', 'Me_Nva', 'Cha', '...",1091.454
6956,8515,CCC[C@H]1C(=O)N[C@@H](CC2CCCCC2)C(=O)N(C)[C@@H...,-6.566211,"['L', 'meA', 'Chg', 'dP', 'Me_Nva', 'Cha', 'me...",1051.389
6957,8516,CCC[C@H]1C(=O)N[C@@H](CC2CCCCC2)C(=O)N(C)[C@@H...,-6.199359,"['L', 'meA', 'Cha', 'dP', 'Me_Nva', 'Cha', 'me...",1065.416
6958,8517,CC[C@@H]1c2nnc(o2)[C@H](Cc2ccccc2)NC(=O)CNC(=O...,-6.780000,"['Mono35', 'G', 'L', 'G']",553.664


In [30]:
#Main dataset is sorted based on Molecular weight in descending order
df_main = df.sort_values(by='MolWt', ascending= False)
df_main

,ID,SMILES,Permeability,Sequence,MolWt
876,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.0,"['meA', 'T', 'meA', 'meL', 'F', 'Me_Phe(3-Cl)'...",1777.744
883,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,"['F', 'meL', 'A', 'meL', 'Ser(tBu)', 'meL', 'm...",1773.325
856,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,"['F', 'meF', 'P', 'L', 'Me_Phe(3-Cl)', 'T', 'L...",1745.057
561,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,"['Sar', 'F', 'meL', 'meL', 'F', 'meL', 'L', 'm...",1733.267
884,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,"['Me_Bal', 'F', 'meF', 'meL', 'T', 'meA', 'meL...",1725.281
...,...,...,...,...,...
1854,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.9,"['A', 'L', 'Mono75']",388.512
1853,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,"['A', 'L', 'H2NEt_Phe']",374.485
1898,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,"['dL', 'S', 'Mono85']",370.494
1897,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,"['dS', 'V', 'Mono85']",356.467


In [31]:
df_main['MolWt']

876     1777.744
883     1773.325
856     1745.057
561     1733.267
884     1725.281
          ...   
1854     388.512
1853     374.485
1898     370.494
1897     356.467
1892     342.440
Name: MolWt, Length: 6960, dtype: float64

In [32]:
#Every fifth datapoint is allocated to test set and remaining datapoints to Train set
test_mask = np.arange(len(df_main)) % 5 == 0
test_df = df_main[test_mask]
test_df.shape

(1392, 5)

In [34]:
train_df = df_main[~test_mask]
train_df.shape

(5568, 5)

In [35]:
print(train_df["Permeability"].max())
print(train_df["Permeability"].min())
print(train_df.shape)

-4.0
-9.46
(5568, 5)


In [36]:
print(test_df["Permeability"].max())
print(test_df["Permeability"].min())
print(test_df.shape)

-3.9
-9.41
(1392, 5)


In [38]:
test_df.to_csv("/home/users/akshay/PCPpred/data/Test.csv",index=False)
train_df.to_csv("/home/users/akshay/PCPpred/data/Train.csv",index=False)

In [39]:
train_df

,ID,SMILES,Permeability,Sequence,MolWt
883,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,"['F', 'meL', 'A', 'meL', 'Ser(tBu)', 'meL', 'm...",1773.325
856,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,"['F', 'meF', 'P', 'L', 'Me_Phe(3-Cl)', 'T', 'L...",1745.057
561,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,"['Sar', 'F', 'meL', 'meL', 'F', 'meL', 'L', 'm...",1733.267
884,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,"['Me_Bal', 'F', 'meF', 'meL', 'T', 'meA', 'meL...",1725.281
868,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,"['meA', 'meL', 'meF', 'T', 'meL', 'meL', 'I', ...",1723.309
...,...,...,...,...,...
1855,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,"['A', 'L', 'Mono76']",402.539
1853,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,"['A', 'L', 'H2NEt_Phe']",374.485
1898,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,"['dL', 'S', 'Mono85']",370.494
1897,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,"['dS', 'V', 'Mono85']",356.467


In [40]:
test_df

,ID,SMILES,Permeability,Sequence,MolWt
876,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,"['meA', 'T', 'meA', 'meL', 'F', 'Me_Phe(3-Cl)'...",1777.744
891,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,"['Me_dA', 'meF', 'meA', 'meL', 'meL', 'F', 'me...",1725.281
865,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,"['dV', 'V', 'T', 'meA', 'F', 'meL', 'meF', 'I'...",1701.218
555,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,"['Sar', 'F', 'meL', 'meL', 'F', 'meL', 'L', 'm...",1686.166
889,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,"['Me_dA', 'meF', 'meA', 'meL', 'I', 'meL', 'F'...",1669.173
...,...,...,...,...,...
1867,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,"['A', 'dL', 'Mono78']",430.593
1871,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,"['A', 'dL', 'Mono87']",430.593
4785,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,"['A', 'iBu_Gly', 'F', '5-Ava']",430.549
1899,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,"['P', 'Y', 'Mono85']",430.549


In [41]:
df_symbol = pd.read_csv('/home/users/akshay/PCPpred/data/CycPeptMPDB_Monomer_All (2).csv')
df_symbol

,ID,Symbol,Version,PubChem_CID,IUPAC_Condensed,Compound_Name,IUPAC_Name,Natural_Analog,replaced_SMILES,CXSMILES,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,1,A,1.0,5950,H-Ala-OH,Alanine,(2S)-2-aminopropanoic acid,A,C[C@H](N)C(=O)O,C[C@H](N[*])C([*])=O |$;;;_R1;;_R2;$|,...,0,0,0,0,0,0,0,0,0,0
1,2,dA,1.0,71080,H-D-Ala-OH,D-alanine,(2R)-2-aminopropanoic acid,A,C[C@@H](N)C(=O)O,C[C@@H](N[*])C([*])=O |$;;;_R1;;_R2;$|,...,0,0,0,0,0,0,0,0,0,0
2,3,meA,1.0,5288725,Me-Ala-OH,N-methyl-L-alanine,(2S)-2-(methylamino)propanoic acid,A,CN[C@@H](C)C(=O)O,C[C@H](N(C)[*])C([*])=O |$;;;;_R1;;_R2;$|,...,0,0,0,0,0,0,0,0,0,0
3,4,Me_dA,1.0,92973,Me-D-Ala-OH,N-methyl-D-alanine,(2R)-2-(methylamino)propanoic acid,A,CN[C@H](C)C(=O)O,C[C@@H](N(C)[*])C([*])=O |$;;;;_R1;;_R2;$|,...,0,0,0,0,0,0,0,0,0,0
4,5,Ala(tBu),1.0,194032,H-Ala(tBu)-OH,3-tert-butyl-L-alanine,"(2S)-2-amino-4,4-dimethylpentanoic acid",A,CC(C)(C)C[C@H](N)C(=O)O,CC(C)(C)C[C@H](N[*])C([*])=O |$;;;;;;;_R1;;_R2;$|,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,Mono123,1.2,N.D,N.D,N.D,(2S)-2-[[5-(aminomethyl)furan-2-yl]methyl-meth...,F,CN(Cc1ccc(CN)o1)[C@@H](Cc1ccccc1)C(=O)O,CN(Cc1ccc(CN[*])o1)[C@@H](Cc1ccccc1)C([*])=O |...,...,0,0,0,0,0,0,0,0,0,0
381,382,Mono124,1.2,N.D,N.D,N.D,(2S)-2-[[5-(methylaminomethyl)furan-2-yl]methy...,F,CNCc1ccc(CN[C@@H](Cc2ccccc2)C(=O)O)o1,CN([*])Cc1ccc(CN[C@@H](Cc2ccccc2)C([*])=O)o1 |...,...,0,0,0,0,0,0,0,0,0,0
382,383,Mono125,1.2,N.D,N.D,N.D,(2S)-2-[methyl-[[5-(methylaminomethyl)furan-2-...,F,CNCc1ccc(CN(C)[C@@H](Cc2ccccc2)C(=O)O)o1,CN([*])Cc1ccc(CN(C)[C@@H](Cc2ccccc2)C([*])=O)o...,...,0,0,0,0,0,0,0,0,0,0
383,384,Mono126,1.2,N.D,N.D,N.D,3-[[3-(aminomethyl)phenyl]-[(2-nitrophenyl)met...,X,NCc1cccc(N(Cc2ccccc2[N+](=O)[O-])C(=O)c2cccc(C...,[*]NCc1cccc(N(Cc2ccccc2[N+](=O)[O-])C(=O)c2ccc...,...,0,0,0,0,0,0,0,0,0,0


In [42]:
symbols = df_symbol['Symbol'].to_list()
symbols

['A',
 'dA',
 'meA',
 'Me_dA',
 'Ala(tBu)',
 'Ala(indol-2-yl)',
 'dAla(indol-2-yl)',
 'Me_Ala(indol-2-yl)',
 'Ala(5-Tet)',
 'Abu',
 'dAbu',
 'Me_Abu',
 'Me_dAbu',
 'Me_Abu(morpholino)',
 '2Abz',
 'Aib',
 'Aoc(2)',
 '5-Ava',
 'Bal',
 'Me_Bal',
 'HOCOCH2_Bal',
 'Cys(EtO2H)_NH2',
 'Cha',
 'dCha',
 'Me_Cha',
 'D',
 'meD',
 'Asp_piperidide',
 'Asp(OMe)',
 'Asp(Ph(2-NH2))',
 'dAsp(pyrrol-1-yl)',
 'E',
 'Glu_NH2',
 'Glu(3R-Me)',
 'Glu(OMe)',
 'dGlu(OMe)',
 'F',
 'dF',
 'meF',
 'Me_dF',
 'Phe(4-F)',
 'dPhe(4-F)',
 'Phe(4-CF3)',
 'Phe(4-NO2)',
 'Phe(CHF2)',
 'dPhe(3,4-diF)',
 'Et_Phe',
 'H2NEt_Phe',
 'Me_Phe(3-Cl)',
 'Me_Phe(4-Cl)',
 'Me_Phe(a,b-dehydro)',
 'G',
 'Bn_Gly',
 'Bn(4-Cl)_Gly',
 'Bn(4-OH)_Gly',
 'Bu_Gly',
 'iBu_Gly',
 'Et_Gly',
 'EtOEt_Gly',
 'HOCOCH2_Gly_ol',
 'MeOEt_Gly',
 'NH2Bu_Gly',
 'Pr_Gly',
 'PhEt_Gly',
 'PhPr_Gly',
 'cHexCH2_Gly',
 'isoamyl_Gly',
 'pentyl_Gly',
 '3-pyridylethyl_Gly',
 '2-pyridylmethyl_Gly',
 'd(N->O)Gly(allyl)',
 'GABA',
 'H',
 'Hph',
 'Me_Hph',
 'bHph',
 '

In [43]:
with open('/home/users/akshay/PCPpred/data/monomer_list.txt','w') as file:
    for symbol in symbols:
        file.write(symbol + '\n')

In [44]:
import pandas as pd

df = pd.read_csv('/home/users/akshay/PCPpred/data/Train.csv')

with open('/home/users/akshay/PCPpred/data/Train.smi', 'w') as f:
    for _, row in df.iterrows():
        smiles = row['SMILES']
        id = row['ID']
        f.write(f"{smiles} {id}\n")
print("train.smi file has been created successfully.")

train.smi file has been created successfully.


In [46]:
df = pd.read_csv('/home/users/akshay/PCPpred/data/Test.csv')

with open('/home/users/akshay/PCPpred/data/Test.smi', 'w') as f:
    for _, row in df.iterrows():
        smiles = row['SMILES']
        id = row['ID']
        f.write(f"{smiles} {id}\n")
print("test.smi file has been created successfully.")

test.smi file has been created successfully.
